In [12]:
import os
import sys
import logging
import datetime
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
sys.path.insert(1, r'methods')

from data_methods import getData, standardize
# from dimensionality_reduction import reduce_dimensionality,reduce_with
# from beamSearch import EMM, as_string
# from adjPysubgroup import adjustedBestFirstSearch, adjustedDFS, adjustedApriori
from qualityMeasures import calc_result_bs, calc_result_ps
from interpretabilityMeasures import Feature_Correlation_Scores, DBI_beam, DBI_ps

In [14]:
# Loading the object using dill
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = 'Ionosphere-data-reductions.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    dfs_dict =pickle.load(f)

In [4]:
# datasets = []
# for dataset in dataset:
#     file_name = '-data-reductions.pkl'
#     en_methods = ['vanilla', 'auto_encoder', 'PCA', 'SPCA']
#     for en_method in en_methods:
#         fcs, fcss = Feature_Correlation_Scores(loaded_dict['vanilla'], loaded_dict[en_method])
#         print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

In [15]:
en_methods = ['auto_encoder', 'PCA', 'SPCA']
for en_method in en_methods:
    fcs, fcss = Feature_Correlation_Scores(dfs_dict['vanilla'], dfs_dict[en_method])
    print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

auto_encoder      
 FCS:  0.59     FCSS 0.12
PCA      
 FCS:  0.65     FCSS 0.14
SPCA      
 FCS:  0.72     FCSS 0.19


In [16]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = 'Ionosphere-results-24-10-2024-15-01.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    results_dict =pickle.load(f)

In [17]:
bfs = results_dict['vanilla']['Best-First Search']['results_org']
bfs_en = results_dict['PCA']['Best-First Search']['results_org']

In [40]:
beam_search = results_dict['auto_encoder']['Beam Search']['results_org']

In [41]:
subgroups_n = [i for i in beam_search.get_values()]

In [44]:
df = dfs_dict['auto_encoder']

In [45]:
df[df.eval(str(subgroups_n[1][1]).replace("', '", ' and ').replace("['", '').replace("']", ''))]

,target,cat1,cat2,cat3,cat4,cat5
2,1,-1.020858,-0.445205,-0.504583,-0.345607,1.549966
3,0,-1.576746,-0.206521,2.954386,5.620608,0.415336
5,0,0.017634,-0.184295,0.566512,-0.093011,-0.564498
13,0,-2.123946,-0.676132,-0.576342,1.267080,2.932600
15,0,-1.208429,-0.564756,-0.115476,4.500622,-0.412116
21,0,9.902280,-0.340863,-0.043100,-0.267024,1.276316
24,1,-1.427370,-0.652125,-0.660910,-0.311437,1.873215
31,0,-2.115209,-0.523013,-1.524813,5.309916,2.495234
33,0,-1.072513,-0.752975,-1.913491,-0.484901,0.635703
34,1,-1.415107,-0.218348,-1.256624,0.667305,2.755546


In [38]:
df.eval(str(subgroups_n[0][1]).replace("', '", ' and ').replace("['", '').replace("']", ''))

0       True
1      False
2       True
3      False
4       True
       ...  
346     True
347     True
348     True
349     True
350     True
Length: 351, dtype: bool

In [47]:
str(subgroups_n[1][1])

"['cat2 > -0.8140837550163269', 'cat2 <= -0.17320257425308228']"

In [10]:
index, zeroes = DBI_beam(subgroups_n, dfs_dict['PCA'])

In [11]:
zeroes

268

In [55]:
subgroups = results_dict['PCA']['Apriori']['results_org']
df_en = dfs_dict['PCA']
df = dfs_dict['vanilla']
features = [feature for feature in df.columns if feature != 'target']
centroids = []
subgroup_cohesion = []
for subgroup_index in range(len(subgroups)):
    oper = str(subgroups["subgroup"][subgroup_index])
    oper = oper.replace("AND", "&")
    if oper.find(":") >= 0 :
        newOpers = []
        splitOper = oper.split(" & ")
        for j in range(len(splitOper)-1, -1, -1) :
            dpIndex = splitOper[j].find(":")
            if dpIndex >= 0 :
                attr = splitOper[j][:dpIndex]
                brIndex = splitOper[j].find("[")
                dpIndex2 = splitOper[j].find(":", dpIndex+1)
                lb = splitOper[j][brIndex+1:dpIndex2]
                ub = splitOper[j][dpIndex2+1:-1]
                newOpers.append(attr+">="+lb)
                newOpers.append(attr+"<="+ub)
                del splitOper[j]
        splitOper += newOpers
        oper = " & ".join(splitOper)
    subgroup_df = df[df_en.eval(oper)][features].astype(float)
    print(subgroup_df.head())

    attribute0  attribute1  attribute2  attribute3  attribute4  attribute5  \
12         1.0         0.0     1.00000    -0.16316         1.0    -0.10169   
14         1.0         0.0     1.00000     0.07380         1.0     0.03420   
16         1.0         0.0     0.99645     0.06468         1.0    -0.01236   
20         1.0         0.0     1.00000    -0.00612         1.0    -0.09834   
22         1.0         0.0     0.96071     0.07088         1.0     0.04296   

    attribute6  attribute7  attribute8  attribute9  ...  attribute24  \
12     0.99999    -0.15197     1.00000    -0.19277  ...      0.84080   
14     1.00000    -0.05563     1.00000     0.08764  ...      1.00000   
16     0.97811     0.02498     0.96112     0.02312  ...      0.81329   
20     1.00000    -0.07649     1.00000    -0.10605  ...      0.81874   
22     1.00000     0.09313     0.90169    -0.05144  ...      0.55677   

    attribute25  attribute26  attribute27  attribute28  attribute29  \
12     -0.47137      0.7622

In [23]:
ps_sd_methods = ['Apriori', 'Best-First Search', 'Depth-First Search']

In [38]:
loaded_dict['vanilla']['Beam Search']['running_time_org']

datetime.timedelta(seconds=689, microseconds=623150)

In [48]:
dct_datasets = ['Ionosphere'] # to test <- Remove later
for dataset in dct_datasets:
    df_file_name = dataset+'-data-reductions.pkl'
    results_file_name = dataset+'-results-24-10-2024-15-01.pkl'
    with open(os.path.join(sd_results_path,df_file_name), 'rb') as f:
        dfs_dict =pickle.load(f)
    with open(os.path.join(sd_results_path,results_file_name), 'rb') as f:
        results_dict =pickle.load(f)
    print(dataset, ":    \n")
    en_methods = ['auto_encoder', 'PCA', 'SPCA']
    ps_sd_methods = ['Apriori', 'Best-First Search', 'Depth-First Search']
    for en_method in en_methods:
        fcs, fcss = Feature_Correlation_Scores(dfs_dict['vanilla'], dfs_dict[en_method])
        print(en_method, ":    ", "\n   FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))
        for sd_method in ps_sd_methods:
            dbi = DBI_ps(results_dict[en_method][sd_method]['results_org'], dfs_dict[en_method])
            print("DBI "+sd_method+": ", dbi)
        subgroups_beam = [i for i in results_dict[en_method]['Beam Search']['results_org'].get_values()]
        dbi = DBI_beam(subgroups_beam, dfs_dict[en_method])
        print("DBI Beam Search: ", dbi, "\n")
    print("\n\n")

Ionosphere :    

auto_encoder :     
   FCS:  0.59     FCSS 0.12
DBI Apriori:  (6.430966551016999, 0)


c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\methods\interpretabilityMeasures.py:104: RuntimeWarning: Mean of empty slice.
  avg_distance_to_centroid = np.linalg.norm(subgroup_df-centroid, axis=1).mean()
c:\Users\20193723\Anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\methods\interpretabilityMeasures.py:104: RuntimeWarning: Mean of empty slice.
  avg_distance_to_centroid = np.linalg.norm(subgroup_df-centroid, axis=1).mean()
c:\Users\20193723\Anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretab

DBI Best-First Search:  (5.4405935782753705, 4)


c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\methods\interpretabilityMeasures.py:104: RuntimeWarning: Mean of empty slice.
  avg_distance_to_centroid = np.linalg.norm(subgroup_df-centroid, axis=1).mean()
c:\Users\20193723\Anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\methods\interpretabilityMeasures.py:104: RuntimeWarning: Mean of empty slice.
  avg_distance_to_centroid = np.linalg.norm(subgroup_df-centroid, axis=1).mean()
c:\Users\20193723\Anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretab

DBI Depth-First Search:  (5.480229483981764, 4)
DBI Beam Search:  (3072388789652733.5, 34) 

PCA :     
   FCS:  0.65     FCSS 0.14
DBI Apriori:  (7.308475820444398, 0)
DBI Best-First Search:  (6.663956085266635, 0)
DBI Depth-First Search:  (6.632011546952284, 0)
DBI Beam Search:  (1.2598977010845176e+16, 268) 

SPCA :     
   FCS:  0.72     FCSS 0.19
DBI Apriori:  (7.088910353666094, 0)
DBI Best-First Search:  (7.172361791274351, 0)
DBI Depth-First Search:  (7.106983731853951, 0)
DBI Beam Search:  (7570163105692239.0, 184) 






In [22]:
for i in loaded_dict['vanilla']:
    print(i)

Best-First Search
Depth-First Search
Apriori
Beam Search


In [11]:
df_en = pd.DataFrame(loaded_dict)

ValueError: If using all scalar values, you must pass an index